# Scraps all auctions from a specific auction house#

In [2]:
from BeautifulSoup import BeautifulSoup
import urllib,string,csv,sys,os
import pandas as pd

In [15]:
# Set the fixed parts of the URL
fx_url_s = 'http://en.artron.net/auction/paimai-result.php?page='
fx_url_e = '&jgid=JG1059'

In [129]:
# To delete after testing
# Load page 1 to get total page count and url row indice
datas = urllib.urlopen('http://en.artron.net/auction/paimai-list.php?page=4&zcid=PZ2029244&o=0').read()
s = BeautifulSoup(datas)
tables = s.findAll("table")
# total page count
page = tables[2].findAll('td')[4].string.strip()
page_count = [int(s) for s in page.split() if s.isdigit()][0]
# url rows indice
count = []
print len(tables[2].findAll('td'))
for i in range(1,100):
    if len(tables[2].findAll('td')[i].findAll('a',href=True))>0:
        count.append(i)

27


IndexError: list index out of range

In [51]:
# To delete after testing
datas = urllib.urlopen('http://en.artron.net/auction/paimai-result.php?page=3&jgid=JG1059').read()
s = BeautifulSoup(datas)
tables = s.findAll("table")
len(tables[4].findAll('td'))
#tables[4].findAll('td')[96]
len(tables[4].findAll('td')[97].findAll('a',href=True))

0

In [139]:
def page_scrap(page_num,url_s,url_e,table_idx):
    datas = urllib.urlopen(url_s+page_num+url_e).read()
    s = BeautifulSoup(datas)
    tables = s.findAll("table")
    td_count = len(tables[table_idx].findAll('td'))
    return td_count

In [159]:
def cleaning(urls,remove_patten):
    new_urls = []
    new_urls = [x for x in urls if remove_patten in x]
    return new_urls

In [168]:
def get_urls(url_s,url_e,table_idx,clean_str):
    url_pre = 'http://en.artron.net/auction/'
    urls = []
    # Use page 1 to calculate total page count to scrap
    result = ''
    if 'jgid' in url_e:
        result = 'auctions'
    if 'zcid' in url_e:
        result = 'items'
    print "Source url:"+url_s+'1'+url_e
    datas = urllib.urlopen(url_s+'1'+url_e).read()
    s = BeautifulSoup(datas)
    tables = s.findAll("table")
    # total page count
    page = tables[2].findAll('td')[4].string.strip()
    page_count = [int(s) for s in page.split() if s.isdigit()][0]
    # Use page count acquired above to calculate individual row's index
    for p in range(1,page_count+1):
        print "Scrapping page:"+str(p)
        tds = page_scrap(str(p),url_s,url_e,table_idx)
        print "Total td tag counts:" + str(tds)
        datas = urllib.urlopen(url_s+str(p)+url_e).read()
        s = BeautifulSoup(datas)
        tables = s.findAll("table")
        # count is to store the index of td with href only
        count = []
        for i in range(1,tds):
            # Look for a td with href in it
            if len(tables[table_idx].findAll('td')[i].findAll('a',href=True))>0:
                count.append(i)
        for i in count:
            # Get the actual href value in each a tag
            auction_href = tables[table_idx].findAll('td')[i].findAll('a',href=True)[0]['href']
            urls.append(url_pre+auction_href)
    urls = cleaning(urls,clean_str)
    print "There are in total %d %s in this scrap" % (len(urls),result)
    return urls

In [169]:
auctions = get_urls(fx_url_s,fx_url_e,4,'paimai-list.php?zcid=')

Source url:http://en.artron.net/auction/paimai-result.php?page=1&jgid=JG1059
Scrapping page:1
Total td tag counts:96
Scrapping page:2
Total td tag counts:90
Scrapping page:3
Total td tag counts:100
Scrapping page:4
Total td tag counts:52
There are in total 67 auctions in this scrap


In [170]:
auctions[:5]

[u'http://en.artron.net/auction/paimai-list.php?zcid=PZ2029244',
 u'http://en.artron.net/auction/paimai-list.php?zcid=PZ2029245',
 u'http://en.artron.net/auction/paimai-list.php?zcid=PZ2029247',
 u'http://en.artron.net/auction/paimai-list.php?zcid=PZ2027078',
 u'http://en.artron.net/auction/paimai-list.php?zcid=PZ2027079']

# To scrap for each auction items, get_urls() can be used with modified url_fixes#

In [171]:
item_urls = []
for a in new_auc[:10]:
    fx_b = a.split('php?zcid=')[0]+'php?page='
    fx_e = '&zcid='+a.split('php?zcid=')[1]
    items = get_urls(fx_b,fx_e,2,'php?picid=')
    item_urls += new_items

Source url:http://en.artron.net/auction/paimai-list.php?page=1&zcid=PZ2029244
Scrapping page:1
Total td tag counts:103
Scrapping page:2
Total td tag counts:103
Scrapping page:3
Total td tag counts:103
Scrapping page:4
Total td tag counts:27
There are in total 69 items in this scrap
Source url:http://en.artron.net/auction/paimai-list.php?page=1&zcid=PZ2029245
Scrapping page:1
Total td tag counts:103
Scrapping page:2
Total td tag counts:103
Scrapping page:3
Total td tag counts:103
Scrapping page:4
Total td tag counts:103
Scrapping page:5
Total td tag counts:103
Scrapping page:6
Total td tag counts:103
Scrapping page:7
Total td tag counts:103
Scrapping page:8
Total td tag counts:103
Scrapping page:9
Total td tag counts:103
Scrapping page:10
Total td tag counts:103
Scrapping page:11
Total td tag counts:103
Scrapping page:12
Total td tag counts:103
Scrapping page:13
Total td tag counts:103
Scrapping page:14
Total td tag counts:103
Scrapping page:15
Total td tag counts:91
There are in total 

In [172]:
len(item_urls)

690

In [173]:
item_urls[:5]

[u'http://en.artron.net/auction/showpic.php?picid=art5087740001',
 u'http://en.artron.net/auction/showpic.php?picid=art5087740001',
 u'http://en.artron.net/auction/showpic.php?picid=art5087740001',
 u'http://en.artron.net/auction/showpic.php?picid=art5087740002',
 u'http://en.artron.net/auction/showpic.php?picid=art5087740003']